In [1]:
import sys
import inspect
import argh
import six
import logging

import torch
from collections import namedtuple
import torch.nn as nn

import ae_dataset_loader
import vgg
import resnet
import utils
import numpy as np

In [2]:
idx, x, y, is_adv, org_x, org_y, adv_algo_name, adv_algo_metric, model_name_index, tag = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

In [3]:
dataset_file_name = '/home/alonhare/ae_splitted/ae_dataset/data/ae_dataset-full.job'
ds = ae_dataset_loader.load(dataset_file_name)

In [4]:
len(ds)

120477

In [5]:
model_name = 'vgg'

if model_name == 'vgg':
    model = vgg.create_vgg19_model(pretrained=True)
elif model_name == 'resnet':
    model = resnet.create_resnet56_model(pretrained=True)

In [6]:
data = np.array(ds)
data.shape

(120477, 10)

In [7]:
model_name

'vgg'

# Explore Data 

In [8]:
count_adv = 0
adv_algo_names = []
adv_algo_metrics = []
model_names = []
for row in data:
    if(row[is_adv]):
        count_adv+=1
        if(row[adv_algo_name] not in adv_algo_names):
           adv_algo_names.append(row[adv_algo_name])
        if(row[adv_algo_metric] not in adv_algo_metrics):
            adv_algo_metrics.append(row[adv_algo_metric])
    if(row[model_name_index] not in model_names):
        model_names.append(row[model_name_index])
print('number of samples: ', data.shape[0])    
print('number of adv: ', count_adv)
print('adversarial attacks: ', adv_algo_names)
print('adversarial metric: ', adv_algo_metrics)
print('model names', model_names)

number of samples:  120477
number of adv:  70488
adversarial attacks:  ['CarliniL2Method', 'ProjectedGradientDescent', 'CarliniLInfMethod', 'HopSkipJump', 'SpatialAttack', 'GaussianBlurAttack', 'ElasticNet', 'FastGradientMethod', 'AdditiveGaussianNoiseAttack', 'AdditiveUniformNoiseAttack', 'BlendedUniformNoiseAttack', 'LBFGSAttack', 'DeepFool', 'ContrastReductionAttack', 'NewtonFool']
adversarial metric:  ['2', '1', 'inf']
model names [None, 'resnet56', 'vgg19']


# Extract Data

In [10]:
algo_name = 'CarliniL2Method'
metrics = '2'
name = 'vgg19'
vgg19_CWL2 = []
for row in data:
    if(row[is_adv] and row[adv_algo_name] == algo_name and row[adv_algo_metric] == metrics and row[model_name_index] == name):
        vgg19_CWL2.append(row)
len(vgg19_CWL2)

8313

In [11]:
extracted = np.stack(vgg19_CWL2)
extracted.shape

(8313, 10)

In [12]:
x_benign, y_benign = np.stack(extracted[:, org_x], axis=0), np.stack(extracted[:, org_y], axis=0)
x_adv, y_adv = np.stack(extracted[:, x], axis=0), np.stack(extracted[:, y], axis=0)
print(x_benign.shape)
print(y_benign.shape)
print(x_adv.shape)
print(y_adv.shape)

(8313, 3, 32, 32)
(8313,)
(8313, 3, 32, 32)
(8313,)


In [12]:
sub_modules = model.children()
model_layers = next(sub_modules)
classifier_layers = next(sub_modules)

In [13]:
x_in = (data[:2])[:, x]
x_in = np.stack(x_in, axis=0)
x_tensor = torch.tensor(x_in).cuda()

In [14]:
x_tensor.shape

torch.Size([2, 3, 32, 32])

In [15]:
def batch_prediction(layer, x, batch_size=512):
    predictions = []
    for i in range((int)(x.shape[0]/batch_size)+1):
        x_batch = torch.tensor(x[i*batch_size:np.minimum((i+1)*batch_size, x.shape[0])]).cuda()
        prediction = layer(x_batch)
        del x_batch
#         x_batch.cpu().detach()
        predictions.append(prediction.cpu().detach().numpy())
    total =  np.concatenate(predictions, axis=0)
    print(total.shape)
    return total

In [16]:
def save_by_batch(x, layer_index, batch_size = 512):
    for i in range((int)(x.shape[0]/batch_size)+1):
        x_batch = x[i*batch_size:np.minimum((i+1)*batch_size, x.shape[0])]
#         np.save('logits/'+'layer_'+str(layer_index)+'_'+str(i), x_batch)
        np.save('./logits/vgg19/cwL2/adversarial/'+'layer_'+str(layer_index)+'_'+str(i), x_batch)

In [17]:
def get_relu_outputs(model_layers, classifier_layers, x, save_pre):
    layer_index = 0
    number_of_examples = x.shape[0]
    for layer in model_layers:
        x = batch_prediction(layer, x)
        if(isinstance(layer, nn.ReLU) or isinstance(layer, nn.Linear)):
            save_by_batch(x, layer_index)
            layer_index+=1
    x = x.reshape(number_of_examples, -1)
    for layer in classifier_layers:
        x = batch_prediction(layer, x)
        if(isinstance(layer, nn.ReLU) or isinstance(layer, nn.Linear)):       
            save_by_batch(x, layer_index)
            layer_index+=1

In [18]:
# logits = get_relu_outputs(model_layers.module, classifier_layers, torch.tensor(x_benign).cuda(), 'benign_')

In [19]:
get_relu_outputs(model_layers.module, classifier_layers, x_adv, './logits/vgg19/cwL2/adversarial/adv_')

(8313, 64, 32, 32)
(8313, 64, 32, 32)
(8313, 64, 32, 32)
(8313, 64, 32, 32)
(8313, 64, 16, 16)
(8313, 128, 16, 16)
(8313, 128, 16, 16)
(8313, 128, 16, 16)
(8313, 128, 16, 16)
(8313, 128, 8, 8)
(8313, 256, 8, 8)
(8313, 256, 8, 8)
(8313, 256, 8, 8)
(8313, 256, 8, 8)
(8313, 256, 8, 8)
(8313, 256, 8, 8)
(8313, 256, 8, 8)
(8313, 256, 8, 8)
(8313, 256, 4, 4)
(8313, 512, 4, 4)
(8313, 512, 4, 4)
(8313, 512, 4, 4)
(8313, 512, 4, 4)
(8313, 512, 4, 4)
(8313, 512, 4, 4)
(8313, 512, 4, 4)
(8313, 512, 4, 4)
(8313, 512, 2, 2)
(8313, 512, 2, 2)
(8313, 512, 2, 2)
(8313, 512, 2, 2)
(8313, 512, 2, 2)
(8313, 512, 2, 2)
(8313, 512, 2, 2)
(8313, 512, 2, 2)
(8313, 512, 2, 2)
(8313, 512, 1, 1)
(8313, 512)
(8313, 512)
(8313, 512)
(8313, 512)
(8313, 512)
(8313, 512)
(8313, 10)


In [13]:
np.save('./logits/vgg19/cwL2/labels_benign', y_benign)